<a href="https://colab.research.google.com/github/EAsencios/DEEP-LEARING/blob/master/fi_on_text_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
corpus = ['The sky is blue and beautiful.',
          'Love this blue and beautiful sky!',
          'The quick brown fox jumps over the lazy dog.',
          'The brown fox is quick and the blue dog is lazy!',
          'The sky es very blue and the sky is very beautiful today',
          'The dog is lazy but the brown fox is quick!'
]
labels = ['weather', 'weather', 'animal', 'animal', 'weather', 'animal']
corpus = np.array(corpus)
corpus_df = pd.DataFrame({'Document':corpus,
                          'Category':labels})
corpus_df = corpus_df[['Document', 'Category']]
corpus_df

,Document,Category
0,The sky is blue and beautiful.,weather
1,Love this blue and beautiful sky!,weather
2,The quick brown fox jumps over the lazy dog.,animal
3,The brown fox is quick and the blue dog is lazy!,animal
4,The sky es very blue and the sky is very beaut...,weather
5,The dog is lazy but the brown fox is quick!,animal


In [4]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

In [5]:
def normalize_document(doc):
  # lower case and remove special characters\whitespaces
  doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I)
  doc = doc.lower()
  doc = doc.strip()
  # tokenize document
  tokens = wpt.tokenize(doc)
  # filter stopwords out of document
  filtered_tokens = [token for token in tokens if token not in stop_words]
  # re-create document from filtered tokens
  doc = ' '.join(filtered_tokens)
  return doc

In [6]:
normalize_corpus = np.vectorize(normalize_document)
norm_corpus = normalize_corpus(corpus)
norm_corpus

array(['sky blue beautiful', 'love blue beautiful sky',
       'quick brown fox jumps lazy dog', 'brown fox quick blue dog lazy',
       'sky es blue sky beautiful today', 'dog lazy brown fox quick'],
      dtype='<U31')

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df=0, max_df=1.)
cv_matrix = cv.fit_transform(norm_corpus)
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0],
       [0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0],
       [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 2, 1],
       [0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0]])

In [8]:
vocab = cv.get_feature_names()
pd.DataFrame(cv_matrix, columns=vocab)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,beautiful,blue,brown,dog,es,fox,jumps,lazy,love,quick,sky,today
0,1,1,0,0,0,0,0,0,0,0,1,0
1,1,1,0,0,0,0,0,0,1,0,1,0
2,0,0,1,1,0,1,1,1,0,1,0,0
3,0,1,1,1,0,1,0,1,0,1,0,0
4,1,1,0,0,1,0,0,0,0,0,2,1
5,0,0,1,1,0,1,0,1,0,1,0,0


In [9]:
bv = CountVectorizer(ngram_range=(2,2))
bv_matrix = bv.fit_transform(norm_corpus)
bv_matrix = bv_matrix.toarray()
vocab = bv.get_feature_names()
pd.DataFrame(bv_matrix, columns=vocab)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,beautiful sky,beautiful today,blue beautiful,blue dog,blue sky,brown fox,dog lazy,es blue,fox jumps,fox quick,jumps lazy,lazy brown,lazy dog,love blue,quick blue,quick brown,sky beautiful,sky blue,sky es
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0
3,0,0,0,1,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0
4,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1
5,0,0,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0, max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(norm_corpus)
tv_matrix = tv_matrix.toarray()

vocab = tv.get_feature_names()
pd.DataFrame(np.around(tv_matrix, 2), columns=vocab)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,beautiful,blue,brown,dog,es,fox,jumps,lazy,love,quick,sky,today
0,0.60,0.52,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.60,0.00
1,0.46,0.39,0.00,0.00,0.00,0.00,0.00,0.00,0.66,0.00,0.46,0.00
2,0.00,0.00,0.38,0.38,0.00,0.38,0.54,0.38,0.00,0.38,0.00,0.00
3,0.00,0.36,0.42,0.42,0.00,0.42,0.00,0.42,0.00,0.42,0.00,0.00
4,0.32,0.27,0.00,0.00,0.46,0.00,0.00,0.00,0.00,0.00,0.64,0.46
5,0.00,0.00,0.45,0.45,0.00,0.45,0.00,0.45,0.00,0.45,0.00,0.00


In [11]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(tv_matrix)
similarity_df = pd.DataFrame(similarity_matrix)
similarity_df

,0,1,2,3,4,5
0,1.000000,0.753128,0.000000,0.185447,0.717486,0.000000
1,0.753128,1.000000,0.000000,0.139665,0.540359,0.000000
2,0.000000,0.000000,1.000000,0.784362,0.000000,0.839987
3,0.185447,0.139665,0.784362,1.000000,0.097425,0.933779
4,0.717486,0.540359,0.000000,0.097425,1.000000,0.000000
5,0.000000,0.000000,0.839987,0.933779,0.000000,1.000000


In [12]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=2)
km.fit_transform(similarity_df)
cluster_labels = km.labels_
cluster_labels = pd.DataFrame(cluster_labels, columns=['ClusterLabel'])
pd.concat([corpus_df, cluster_labels], axis=1)

,Document,Category,ClusterLabel
0,The sky is blue and beautiful.,weather,1
1,Love this blue and beautiful sky!,weather,1
2,The quick brown fox jumps over the lazy dog.,animal,0
3,The brown fox is quick and the blue dog is lazy!,animal,0
4,The sky es very blue and the sky is very beaut...,weather,1
5,The dog is lazy but the brown fox is quick!,animal,0


In [13]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=2, max_iter=100, random_state=42)
dt_matrix = lda.fit_transform(tv_matrix)
features = pd.DataFrame(dt_matrix, columns=['T1', 'T2'])
features

,T1,T2
0,0.191131,0.808869
1,0.177271,0.822729
2,0.846406,0.153594
3,0.817973,0.182027
4,0.168328,0.831672
5,0.839354,0.160646


In [14]:
from sklearn.utils.extmath import weighted_mode
tt_matrix = lda.components_
for topic_weights in tt_matrix:
  topic = [(token, weight) for token, weight in zip(vocab, topic_weights)]
  topic = sorted(topic, key=lambda x: -x[1])
  topic = [item for item in topic if item[1] > 0.6]
  print(topic)
  print()

[('brown', 1.7279413066420188), ('dog', 1.7279413066420188), ('fox', 1.7279413066420188), ('lazy', 1.7279413066420188), ('quick', 1.7279413066420188), ('jumps', 1.03310482744227), ('blue', 0.7828939229380227)]

[('sky', 2.1846841668045975), ('beautiful', 1.8668438278226838), ('blue', 1.7557208168256953), ('love', 1.1478159291395924), ('es', 0.9500895007344254), ('today', 0.9500895007344254)]



In [15]:
km = KMeans(n_clusters=2)
km.fit_transform(features)
cluster_labels = km.labels_
cluster_labels = pd.DataFrame(cluster_labels, columns=['ClusterLabel'])
pd.concat([corpus_df, cluster_labels], axis=1)

,Document,Category,ClusterLabel
0,The sky is blue and beautiful.,weather,1
1,Love this blue and beautiful sky!,weather,1
2,The quick brown fox jumps over the lazy dog.,animal,0
3,The brown fox is quick and the blue dog is lazy!,animal,0
4,The sky es very blue and the sky is very beaut...,weather,1
5,The dog is lazy but the brown fox is quick!,animal,0


In [16]:
from gensim.models import word2vec

wpt = nltk.WordPunctTokenizer()
tokenized_corpus = [wpt.tokenize(document) for document in norm_corpus]

# Sent values for various parameters
feature_size = 10       # Word vector dimensionality
window_context = 10     # Context window size
min_word_count = 1      # Minimum word count
sample = 1e-3           # Downsample setting for frequent words

w2v_model = word2vec.Word2Vec(tokenized_corpus, size=feature_size, window=window_context, min_count=min_word_count, sample=sample)
w2v_model.wv['sky']

array([-0.01519162,  0.0047406 ,  0.04953602, -0.03473093,  0.00791535,
       -0.02844507,  0.03475524, -0.03061048,  0.02909456, -0.01777873],
      dtype=float32)

In [73]:
def average_word_vectors(words, model, vocabulary, num_features):
  feature_vector = np.zeros((num_features,), dtype='float64')
  nwords = 0.

  for word in words:
    if word in vocabulary:
      nwords = nwords + 1
      feature_vector = np.add(feature_vector, model[word])
    if nwords:
      feature_vector = np.divide(feature_vector, nwords)
  return feature_vector

def average_word_vectorizer (corpus, model, num_features):
  vocabulary = set(model.wv.index2word)
  feature = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features) for tokenized_sentence in corpus]
  return np.array(features)

In [74]:
w2v_feature_array = average_word_vectorizer(corpus=tokenized_corpus, model=w2v_model, num_features=feature_size)
pd.DataFrame(w2v_feature_array)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


,0,1
0,0.191131,0.808869
1,0.177271,0.822729
2,0.846406,0.153594
3,0.817973,0.182027
4,0.168328,0.831672
5,0.839354,0.160646


In [99]:
model=w2v_model
vocabulary = set(model.wv.index2word)

for words in tokenized_corpus:
  feature = average_word_vectors(words, model, vocabulary, feature_size)
  print(feature)


[-0.00672398  0.015876    0.02554333 -0.00872257 -0.01791599  0.00206333
 -0.00438368 -0.01019873  0.01023355  0.00885859]
[-0.00477311  0.00637955  0.01629773 -0.00760203 -0.0047057  -0.00631701
  0.00625686 -0.00928721  0.00979432  0.00011209]
[ 0.00219534  0.00454994 -0.00079512 -0.00344586  0.00087124 -0.00048556
  0.00069627 -0.00663379  0.00400505  0.00309187]
[ 0.00657605  0.00261476 -0.00183012 -0.00511009  0.00350957 -0.00238968
 -0.00515339  0.00452479  0.0033376  -0.00449546]
[-0.00677932 -0.00510618  0.00293672  0.00275858 -0.00803654  0.00156144
  0.00674076  0.00688408  0.00106759  0.00345936]
[ 0.01019655  0.00389436 -0.00354634 -0.00680969  0.00799023 -0.00376872
 -0.01247931 -0.01122769  0.00804642 -0.0013702 ]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
